Version of the corresponding [training notebook](https://www.kaggle.com/code/sasaleaf/csiro-pytorch-baseline-train): **Version 10**
# 🌱 PyTorch CNN Baseline

This notebook presents a **CNN** baseline built with **PyTorch**,
based on **EfficientNet v0**.
The final head is modified to output **3** predictions,
and the model is fine-tuned for this task.

---
## 🎯 Target Structure
The neural network directly predicts 3 out of 5 target variables:
- `Dry_Clover_g`
- `Dry_Dead_g`
- `GDM_g`

The remaining 2 targets are derived using the following relationships:
- `Dry_Green_g` = `GDM_g` - `Dry_Clover_g`
- `Dry_Total_g` = `GDM_g` + `Dry_Dead_g`

---
💬 Note:
If you notice anything unclear or have suggestions for improvement,
please feel free to leave a comment!

In [ ]:
import os
import gc
import random
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2
from efficientnet_pytorch import model as enet
import matplotlib.pyplot as plt

# Config

In [ ]:
# ======== 1. Config ========

class CFG:
    # path
    data_dir = "/kaggle/input/csiro-biomass/"
    train_csv_path = os.path.join(data_dir, "train.csv")
    test_csv_path = os.path.join(data_dir, "test.csv")
    sample_sub_path = os.path.join(data_dir, "sample_submission.csv")
    
    # CV
    n_folds = 5
    seed = 42
    
    # model
    model_name = "efficientnet-b0"
    pretrained = False
    pretrained_weights_path = "/kaggle/input/efficientnet-pytorch/efficientnet-b0-08094119.pth"
    best_model_dir = "/kaggle/input/csiro-pytorch-baseline-train/"
    
    # image
    img_size_h = 256
    img_size_w = 512
    in_chans = 3
    
    # train
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    epochs = 150
    batch_size = 32
    lr = 1e-3
    eta_min = 1e-5
    weight_decay = 1e-6
    
    # target columns
    target_cols = [
        "Dry_Clover_g",
        "Dry_Dead_g",
        "Dry_Green_g",
        "Dry_Total_g",
        "GDM_g"
    ]
    n_targets = 3 # Dry_Clover_g, Dry_Dead_g, GDM_g

# Utility

In [ ]:
# ======== 2. Utility ========

def seed_everything(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def weighted_r2_score(y_true: np.ndarray, y_pred: np.ndarray):
    """
    Metric
    y_true, y_pred: shape (N, 5)
    """
    weights = np.array([0.1, 0.1, 0.1, 0.2, 0.5])
    r2_scores = []
    
    for i in range(y_true.shape[1]):
        y_t = y_true[:, i]
        y_p = y_pred[:, i]
        ss_res = np.sum((y_t - y_p) ** 2)
        ss_tot = np.sum((y_t - np.mean(y_t)) ** 2)
        r2 = 1 - ss_res / ss_tot if ss_tot > 0 else 0.0
        r2_scores.append(r2)
        
    r2_scores = np.array(r2_scores)
    weighted_r2 = np.sum(r2_scores * weights) / np.sum(weights)
    return weighted_r2, r2_scores

seed_everything(CFG.seed)

# Preprocessing

In [ ]:
# ======== 3. Preprocessing ========

def get_processed_data(cfg):
    """
    'long' -> 'wide'
    """
    train_df = pd.read_csv(cfg.train_csv_path)
    
    # unique id (ex: ID1011485656)
    train_df["image_id"] = train_df["image_path"].apply(lambda x: x.split('/')[-1].split('.')[0])
    
    # pivot
    train_pivot = train_df.pivot(
        index="image_id", 
        columns="target_name", 
        values="target"
    ).reset_index()
    
    meta_df = train_df.drop_duplicates(subset="image_id").drop(
        columns=["sample_id", "target_name", "target"]
    )
    
    train_processed_df = meta_df.merge(train_pivot, on="image_id", how="left")
    
    # CV fold
    kf = KFold(n_splits=cfg.n_folds, shuffle=True, random_state=cfg.seed)
    train_processed_df["fold"] = -1
    for fold, (train_idx, val_idx) in enumerate(kf.split(train_processed_df)):
        train_processed_df.loc[val_idx, "fold"] = fold
        
    return train_processed_df

# Dataset

In [ ]:
# ======== 4. Dataset & Augmentations ========

def get_transforms(is_train):
    """Augmentation"""
    if is_train:
        return A.Compose([
            A.Resize(CFG.img_size_h, CFG.img_size_w),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.Rotate(limit=30, p=0.5),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2(),
        ])
    else:
        return A.Compose([
            A.Resize(CFG.img_size_h, CFG.img_size_w),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2(),
        ])

class BiomassDataset(Dataset):
    def __init__(self, df, transforms=None, is_test=False):
        self.df = df
        self.image_paths = df["image_path"].values
        self.transforms = transforms
        self.is_test = is_test
        
        if not self.is_test:
            self.targets = df[CFG.target_cols].values

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        full_path = os.path.join(CFG.data_dir, image_path)
        
        try:
            image = cv2.imread(full_path)
            if image is None:
                raise FileNotFoundError(f"Image not found at {full_path}")
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        except Exception as e:
            print(f"Error loading image {full_path}: {e}")
            # dummy
            image = np.zeros((CFG.img_size_h, CFG.img_size_w, 3), dtype=np.uint8)

        # transform
        if self.transforms:
            image = self.transforms(image=image)["image"]
        
        if self.is_test:
            return image
        else:
            target = torch.tensor(self.targets[idx], dtype=torch.float32)
            return image, target

# Model

In [ ]:
# ======== 5. Model ========

class BiomassModel(nn.Module):
    def __init__(self, model_name=CFG.model_name, pretrained=CFG.pretrained, n_targets=CFG.n_targets):
        super().__init__()
        self.model = enet.EfficientNet.from_pretrained(
            model_name, 
            weights_path=CFG.pretrained_weights_path,
            in_channels=CFG.in_chans
        )
        
        # number of output features
        in_features = self.model._fc.in_features
        
        # head
        self.model._fc = nn.Linear(in_features, n_targets)

    def forward(self, x):
        output = self.model(x)
        return output

# Train / Valdation

In [ ]:
# ======== 3 <-> 5 target transform ========

def get_loss_targets(targets_5):
    """
    Extracts 3 target values (B, 3) for loss computation from the 5 ground truth targets (B, 5).

    Args:
        targets_5 (torch.Tensor): Tensor of shape (B, 5) containing 
            [Clover, Dead, Green, Total, GDM].

    Returns:
        torch.Tensor: Tensor of shape (B, 3) containing the selected ground truth targets 
            [Clover_true, Dead_true, GDM_true].
    """
    
    # Index 0: Dry_Clover_g
    # Index 1: Dry_Dead_g
    # Index 4: GDM_g
    loss_targets_3 = torch.stack(
        [
            targets_5[:, 0], # Clover
            targets_5[:, 1], # Dead
            targets_5[:, 4]  # GDM
        ],
        dim=1
    )
    return loss_targets_3

def expand_predictions_torch(preds_3):
    """
    [torch] Expand the 3 NN predictions (N, 3) to 5 predictions (N, 5).
    """
    # clip
    P_Clover = torch.clamp(preds_3[:, 0], min=0)
    P_Dead = torch.clamp(preds_3[:, 1], min=0)
    P_GDM = torch.clamp(preds_3[:, 2], min=0)
    
    # Compute derived targets based on constraints.
    P_Green = torch.clamp(P_GDM - P_Clover, min=0)
    P_Total = P_GDM + P_Dead
    
    preds_5 = torch.stack(
        [
            P_Clover, # Index 0
            P_Dead,   # Index 1
            P_Green,  # Index 2
            P_Total,  # Index 3
            P_GDM     # Index 4
        ],
        dim=1
    )
    return preds_5

def expand_predictions_np(preds_3):
    """
    [Numpy] Expand the three NN predictions (N, 3) to five predictions (N, 5).
    """
    P_Clover = np.clip(preds_3[:, 0], a_min=0, a_max=None)
    P_Dead = np.clip(preds_3[:, 1], a_min=0, a_max=None)
    P_GDM = np.clip(preds_3[:, 2], a_min=0, a_max=None)
    
    P_Green = np.clip(P_GDM - P_Clover, a_min=0, a_max=None)
    P_Total = P_GDM + P_Dead
    
    preds_5 = np.stack(
        [P_Clover, P_Dead, P_Green, P_Total, P_GDM],
        axis=1
    )
    return preds_5


In [ ]:
# ======== 6. Train / Validation ========

def train_fn(model, dataloader, optimizer, criterion, device):
    """Training function for 1 epoch."""
    model.train()
    total_loss = 0
    
    for images, targets in dataloader:
        images = images.to(device)
        targets_5 = targets.to(device) # (B, 5)
        
        optimizer.zero_grad()
        outputs_3 = model(images) # (B, 3) [Clover_pred, Dead_pred, GDM_pred]
        targets_3_for_loss = get_loss_targets(targets_5) # (B, 3) [Clover_true, Dead_true, GDM_true]
        loss = criterion(outputs_3, targets_3_for_loss)
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
    return total_loss / len(dataloader)

def val_fn(model, dataloader, criterion, device):
    """Eval function for 1 epoch."""
    model.eval()
    total_loss = 0
    all_targets_5_np = []
    all_preds_5_np = []
    
    with torch.no_grad():
        for images, targets in dataloader:
            images = images.to(device)
            targets_5 = targets.to(device) # (B, 5)
            
            outputs_3 = model(images) # (B, 3) [Clover_pred, Dead_pred, GDM_pred]
            targets_3_for_loss = get_loss_targets(targets_5)
            loss = criterion(outputs_3, targets_3_for_loss)
            
            total_loss += loss.item()
            preds_5 = expand_predictions_torch(outputs_3)
            
            all_targets_5_np.append(targets_5.cpu().numpy())
            all_preds_5_np.append(preds_5.cpu().numpy())
            
    val_loss = total_loss / len(dataloader)
    
    # NumPy
    y_true_5 = np.concatenate(all_targets_5_np, axis=0)
    y_pred_5 = np.concatenate(all_preds_5_np, axis=0)
    
    # Metric
    weighted_r2, _ = weighted_r2_score(y_true_5, y_pred_5)
    
    return val_loss, weighted_r2

# Running

In [ ]:
# ======== 7. CV Training ========

def run_training():
    print(f"Device: {CFG.device}")
    
    # 1. data
    train_df = get_processed_data(CFG)
    oof_predictions = np.zeros((len(train_df), len(CFG.target_cols))) # (N, 5)
    
    for fold in range(CFG.n_folds):
        print(f"\n======== FOLD {fold+1} / {CFG.n_folds} ========")
        
        # 2. fold
        train_fold_df = train_df[train_df["fold"] != fold].reset_index(drop=True)
        val_fold_df_with_index = train_df[train_df["fold"] == fold]
        val_indices = val_fold_df_with_index.index
        val_fold_df = val_fold_df_with_index.reset_index(drop=True)
        
        # 3. Dataset & DataLoader
        train_dataset = BiomassDataset(train_fold_df, transforms=get_transforms(is_train=True))
        val_dataset = BiomassDataset(val_fold_df, transforms=get_transforms(is_train=False))
        
        train_loader = DataLoader(
            train_dataset, batch_size=CFG.batch_size, shuffle=True, 
            num_workers=os.cpu_count(), pin_memory=True
        )
        val_loader = DataLoader(
            val_dataset, batch_size=CFG.batch_size * 2, shuffle=False, 
            num_workers=os.cpu_count(), pin_memory=True
        )
        
        # 4. model
        model = BiomassModel().to(CFG.device)
        criterion = nn.MSELoss() 
        optimizer = optim.AdamW(model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay)
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=CFG.epochs, eta_min=CFG.eta_min)
        
        # 5. training
        best_val_score = -np.inf
        model_path = f"model_fold_{fold}.pth"

        history = {
            'train_loss': [],
            'val_loss': []
        }
        
        for epoch in range(CFG.epochs):
            train_loss = train_fn(model, train_loader, optimizer, criterion, CFG.device)
            val_loss, val_score = val_fn(model, val_loader, criterion, CFG.device)

            history['train_loss'].append(train_loss)
            history['val_loss'].append(val_loss)

            print(f"Epoch {epoch+1}/{CFG.epochs} - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val R2: {val_score:.4f}")
            
            scheduler.step()
            
            # best score
            if val_score > best_val_score:
                best_val_score = val_score
                torch.save(model.state_dict(), model_path)
                print(f"Best model saved to {model_path} (Score: {best_val_score:.4f})")
                
        # viz
        plt.figure(figsize=(10, 4))
        plt.plot(history['train_loss'], label='Train Loss')
        plt.plot(history['val_loss'], label='Validation Loss')
        plt.title(f'Fold {fold+1} - Train & Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(True)
        plt.show()

        # save OOF predictions.
        print(f"Loading best model for OOF prediction from {model_path}")
        model.load_state_dict(torch.load(model_path))
        
        val_loader_for_oof = DataLoader(
            val_dataset, batch_size=CFG.batch_size * 2, shuffle=False, 
            num_workers=os.cpu_count(), pin_memory=True
        )
        model.eval()
        fold_preds_list_3 = []
        with torch.no_grad():
             for images, _ in val_loader_for_oof:
                images = images.to(CFG.device)
                preds_3 = model(images) # (B, 3)
                fold_preds_list_3.append(preds_3.cpu().numpy())
        
        oof_fold_preds_3 = np.concatenate(fold_preds_list_3, axis=0) # (N_val, 3)
        oof_fold_preds_5 = expand_predictions_np(oof_fold_preds_3)
        oof_predictions[val_indices] = oof_fold_preds_5

        del model, train_dataset, val_dataset, train_loader, val_loader
        gc.collect()
        torch.cuda.empty_cache()

    # Final OOF score
    oof_score, oof_scores_by_target = weighted_r2_score(train_df[CFG.target_cols].values, oof_predictions)
    print(f"\n======== CV Finished ========")
    print(f"Overall OOF Weighted R2 Score: {oof_score:.4f}")
    print("OOF R2 by Target:")
    for i, col in enumerate(CFG.target_cols):
        print(f"  - {col}: {oof_scores_by_target[i]:.4f}")

In [ ]:
# ======== 8. Inference ========

def run_inference():
    print("\n======== Starting Inference ========")
    
    # 1. data
    test_df = pd.read_csv(CFG.test_csv_path)
    test_unique_df = test_df.drop_duplicates(subset="image_path").reset_index(drop=True)
    
    test_dataset = BiomassDataset(
        test_unique_df, 
        transforms=get_transforms(is_train=False), 
        is_test=True
    )
    test_loader = DataLoader(
        test_dataset, batch_size=CFG.batch_size * 2, shuffle=False,
        num_workers=os.cpu_count(), pin_memory=True
    )
    
    # 2. Prediction using 5-fold models
    all_fold_preds = []
    for fold in range(CFG.n_folds):
        print(f"Predicting with Fold {fold+1}...")
        model_path = os.path.join(CFG.best_model_dir, f"model_fold_{fold}.pth")
        
        model = BiomassModel().to(CFG.device)
        try:
            model.load_state_dict(torch.load(model_path))
        except FileNotFoundError:
            print(f"Warning: Model file {model_path} not found. Skipping fold {fold+1}.")
            continue
            
        model.eval()
        
        fold_preds = []
        with torch.no_grad():
            for images in test_loader:
                images = images.to(CFG.device)
                outputs = model(images)
                fold_preds.append(outputs.cpu().numpy())
        
        all_fold_preds.append(np.concatenate(fold_preds, axis=0))
    
    if not all_fold_preds:
        print("Error: No models were loaded. Cannot perform inference.")
        return

    # 3. average
    # (n_folds, n_test_images, n_targets) -> (n_test_images, n_targets)
    avg_preds_3 = np.mean(all_fold_preds, axis=0) # (n_test_images, 3)
    avg_preds_5 = expand_predictions_np(avg_preds_3) # (n_test_images, 5)
    
    # 4. submission
    
    # 'wide'
    preds_df = pd.DataFrame(avg_preds_5, columns=CFG.target_cols)
    test_unique_df = pd.concat([test_unique_df, preds_df], axis=1)
    
    # -> 'long'
    test_pred_long_df = test_unique_df.melt(
        id_vars=["image_path"], 
        value_vars=CFG.target_cols,
        var_name="target_name",
        value_name="target"
    )

    submission_df = test_df[["sample_id", "image_path", "target_name"]].merge(
        test_pred_long_df,
        on=["image_path", "target_name"],
        how="left"
    )
    
    final_submission = submission_df[["sample_id", "target"]].copy()

    # sanitize
    final_submission["target"] = final_submission["target"].fillna(0)
    final_submission["target"] = final_submission["target"].replace([np.inf, -np.inf], 0)
    final_submission["target"] = final_submission["target"].clip(lower=0)
    
    # submission
    final_submission.to_csv("submission.csv", index=False)
    print("Inference complete. submission.csv saved.")
    print(final_submission.head())

# Main

In [ ]:
# run_training()

In [ ]:
run_inference()